In [1]:
from pyspark.sql import SparkSession
from pprint import pprint
# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("mei_wu_part-b")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
#        .config('spark.executor.cores', 2)\


# Old API (RDD)
spark_context = spark_session.sparkContext

data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.87:9000/parking-citations.csv')\
    .cache()

In [2]:
# B.1
data_frame.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|  

In [17]:
# B.2
data_frame.rdd.getNumPartitions()

11

In [52]:
# B.3
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [20]:
# B.4
data_frame.count()

9881842

In [70]:
# B.5
data_frame.drop("Agency Description", "Agency", "Route").take(5)

[Row(Ticket number='1103341116', Issue Date='2015-12-21T00:00:00.000', Issue time='1251', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='200304', VIN=None, Make='HOND', Body Style='PA', Color='GY', Location='13147 WELBY WAY', Violation code='4000A1', Violation Description='NO EVIDENCE OF REG', Fine amount='50', Latitude='99999', Longitude='99999', Color Description=None, Body Style Description=None),
 Row(Ticket number='1103700150', Issue Date='2015-12-21T00:00:00.000', Issue time='1435', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201512', VIN=None, Make='GMC', Body Style='VN', Color='WH', Location='525 S MAIN ST', Violation code='4000A1', Violation Description='NO EVIDENCE OF REG', Fine amount='50', Latitude='99999', Longitude='99999', Color Description=None, Body Style Description=None),
 Row(Ticket number='1104803000', Issue Date='2015-12-21T00:00:00.000', Issue time='2055', Meter Id=None, Marked Time=None, RP State Plate='CA',

In [161]:
import pyspark
from pyspark.sql.types import DoubleType

# B.6
data_frame.withColumn("Fine amount", data_frame["Fine amount"]\
                      .cast(DoubleType())).groupBy().avg('Fine amount').show()

+----------------+
|avg(Fine amount)|
+----------------+
|70.1855354220642|
+----------------+



In [186]:
# B.7
data_frame.groupBy('Make').count().orderBy('count', ascending = False).show(10)

+----+-------+
|Make|  count|
+----+-------+
|TOYT|1633266|
|HOND|1113834|
|FORD| 860828|
|NISS| 709250|
|CHEV| 674422|
| BMW| 450909|
|MERZ| 402126|
|VOLK| 335618|
|HYUN| 304934|
|DODG| 290979|
+----+-------+
only showing top 10 rows



In [71]:
# B.8
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White',
'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

def abrv_to_long(c):
    if c in COLORS:
        return COLORS[c]
    else:
        return c

# User-defined function. 
udf_abrv_to_long = udf(abrv_to_long, StringType())

data_frame_with_color_long = data_frame.withColumn("Color long", udf_abrv_to_long("color"))

data_frame_with_color_long.cache()

data_frame_with_color_long.filter(data_frame_with_color_long['Make'] == 'HOND')\
.groupBy('color long')\
.count()\
.orderBy('count', ascending = False).show(1)

+----------+------+
|color long| count|
+----------+------+
|      Gray|266135|
+----------+------+
only showing top 1 row



In [72]:
spark_session.stop()